<a href="https://colab.research.google.com/github/NSkuhala/Test1/blob/master/MLC7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##### Chapter 7: Neural Networks and Support Vector Machines -------------------

##### Part 1: Neural Networks -------------------
## Example: Modeling the Strength of Concrete  ----


In [0]:
## Step 2: Exploring and preparing the data ----
# read in data and examine structure
concrete <- read.csv("Chapter07/concrete.csv")
str(concrete)


In [0]:
# custom normalization function
normalize <- function(x) { 
  return((x - min(x)) / (max(x) - min(x)))
}

In [0]:
# apply normalization to entire data frame
concrete_norm <- as.data.frame(lapply(concrete, normalize))

In [0]:
# confirm that the range is now between zero and one
summary(concrete_norm$strength)

In [0]:
# compared to the original minimum and maximum
summary(concrete$strength)

In [0]:
# create training and test data
concrete_train <- concrete_norm[1:773, ]
concrete_test <- concrete_norm[774:1030, ]

In [0]:
## Step 3: Training a model on the data ----
# train the neuralnet model
library(neuralnet)

In [0]:
# simple ANN with only a single hidden neuron
set.seed(12345) # to guarantee repeatable results
concrete_model <- neuralnet(formula = strength ~ cement + slag +
                              ash + water + superplastic + 
                              coarseagg + fineagg + age,
                              data = concrete_train)


In [0]:
# visualize the network topology
plot(concrete_model)


In [0]:
## Step 4: Evaluating model performance ----
# obtain model results
model_results <- compute(concrete_model, concrete_test[1:8])

In [0]:
# obtain predicted strength values
predicted_strength <- model_results$net.result

In [0]:
# examine the correlation between predicted and actual values
cor(predicted_strength, concrete_test$strength)

In [0]:
## Step 5: Improving model performance ----
# a more complex neural network topology with 5 hidden neurons
set.seed(12345) # to guarantee repeatable results
concrete_model2 <- neuralnet(strength ~ cement + slag +
                               ash + water + superplastic + 
                               coarseagg + fineagg + age,
                               data = concrete_train, hidden = 5)


In [0]:
# plot the network
plot(concrete_model2)

In [0]:
# evaluate the results as we did before
model_results2 <- compute(concrete_model2, concrete_test[1:8])
predicted_strength2 <- model_results2$net.result
cor(predicted_strength2, concrete_test$strength)

In [0]:
# an EVEN MORE complex neural network topology with two hidden layers and custom activation function


In [0]:
# create a custom softplus activation function
softplus <- function(x) { log(1 + exp(x)) }

set.seed(12345) # to guarantee repeatable results
concrete_model3 <- neuralnet(strength ~ cement + slag +
                               ash + water + superplastic + 
                               coarseagg + fineagg + age,
                             data = concrete_train, hidden = c(5, 5), act.fct = softplus)


In [0]:
# plot the network
plot(concrete_model3)

In [0]:
# evaluate the results as we did before
model_results3 <- compute(concrete_model3, concrete_test[1:8])
predicted_strength3 <- model_results3$net.result
cor(predicted_strength3, concrete_test$strength)

In [0]:
# note that the predicted and actual values are on different scales
strengths <- data.frame(
  actual = concrete$strength[774:1030],
  pred = predicted_strength3
)

head(strengths, n = 3)


In [0]:
# this doesn't change the correlations (but would affect absolute error)
cor(strengths$pred, strengths$actual)


In [0]:
# create an unnormalize function to reverse the normalization
unnormalize <- function(x) { 
  return((x * (max(concrete$strength)) -
          min(concrete$strength)) + min(concrete$strength))
}

strengths$pred_new <- unnormalize(strengths$pred)
strengths$error <- strengths$pred_new - strengths$actual

head(strengths, n = 3)

cor(strengths$pred_new, strengths$actual)


In [0]:
##### Part 2: Support Vector Machines -------------------
## Example: Optical Character Recognition ----


In [0]:
## Step 2: Exploring and preparing the data ----
# read in data and examine structure
letters <- read.csv("letterdata.csv")
str(letters)


In [0]:
# divide into training and test data
letters_train <- letters[1:16000, ]
letters_test  <- letters[16001:20000, ]

In [0]:
## Step 3: Training a model on the data ----
# begin by training a simple linear SVM
library(kernlab)
letter_classifier <- ksvm(letter ~ ., data = letters_train,
                          kernel = "vanilladot")


In [0]:
# look at basic information about the model
letter_classifier


In [0]:
## Step 4: Evaluating model performance ----
# predictions on testing dataset
letter_predictions <- predict(letter_classifier, letters_test)

head(letter_predictions)

table(letter_predictions, letters_test$letter)

In [0]:
# look only at agreement vs. non-agreement
# construct a vector of TRUE/FALSE indicating correct/incorrect predictions
agreement <- letter_predictions == letters_test$letter
table(agreement)
prop.table(table(agreement))

In [0]:
## Step 5: Improving model performance ----

# change to a RBF kernel
RNGversion("3.5.2") # use an older random number generator to match the book
set.seed(12345)
letter_classifier_rbf <- ksvm(letter ~ ., data = letters_train, kernel = "rbfdot")
letter_predictions_rbf <- predict(letter_classifier_rbf, letters_test)

agreement_rbf <- letter_predictions_rbf == letters_test$letter
table(agreement_rbf)
prop.table(table(agreement_rbf))

In [0]:
# test various values of the cost parameter
cost_values <- c(1, seq(from = 5, to = 40, by = 5))

RNGversion("3.5.2") # use an older random number generator to match the book
accuracy_values <- sapply(cost_values, function(x) {
  set.seed(12345)
  m <- ksvm(letter ~ ., data = letters_train,
            kernel = "rbfdot", C = x)
  pred <- predict(m, letters_test)
  agree <- ifelse(pred == letters_test$letter, 1, 0)
  accuracy <- sum(agree) / nrow(letters_test)
  return (accuracy)
})

plot(cost_values, accuracy_values, type = "b")
